In [ ]:
# 匯入相關套件
import os
import time
from datetime import datetime
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# region
def wait_and_click(xpath, timeout=5, use_js=False, use_click=True):
    """
    等待指定 xpath 的元素出現並點擊。

    Args:
        xpath (str): XPath 路徑
        timeout (int): 最多等待秒數
        use_js (bool): 是否用 JavaScript 點擊（預設 False）
    """
    if use_click:
        element = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
    else:
        element = WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located((By.XPATH, xpath))
        )
    if use_js:
        driver.execute_script("arguments[0].click();", element)
    else:
        element.click()


def fuzzy_match(
    outer_attr, outer_keyword, inner_attr=None, inner_keyword=None, click=True
):
    """
    找出內層符合 inner_attr ≈ inner_keyword 的元素，若 click=True 自動 scroll + 點擊。
    忽略空格、標點、大小寫差異。
    """
    global driver
    script = """
        const norm = s => s ? s.replace(/\\W+/g, "").toLowerCase() : "";
        const outerKey = norm(arguments[0]);
        const innerKey = norm(arguments[1]);
        const outerAttr = arguments[2];
        const innerAttr = arguments[3];
        const shouldClick = arguments[4];

        const outer = Array.from(document.querySelectorAll(`[${outerAttr}]`))
            .find(el => norm(el.getAttribute(outerAttr)).includes(outerKey));
        
        if (!outer) return null;

        if (!innerAttr) {
            if (shouldClick) {
                outer.scrollIntoView({block: "center"});
                setTimeout(() => outer.click(), 100);
            }
            return shouldClick ? null : outer;
        }

        const matched = Array.from(outer.querySelectorAll(`[${innerAttr}]`))
            .find(el => norm(el.getAttribute(innerAttr)).includes(innerKey));
        
        if (matched && shouldClick) {
            matched.scrollIntoView({block: "center"});
            setTimeout(() => matched.click(), 100);
        }

        return shouldClick ? null : matched;
    """

    return driver.execute_script(
        script, outer_keyword, inner_keyword or "", outer_attr, inner_attr or "", click
    )


# endregion

In [ ]:
# ---------- 初始設定 ----------
load_dotenv()
TARGET_TIME = os.getenv("SHIP_TARGET_TIME")
URL = os.getenv("SHIP_URL")
EMAIL = os.getenv("SHIP_EMAIL")
PASSWORD = os.getenv("SHIP_PASSWORD")
SHOP = os.getenv("SHIP_SHOP")
PRODUCTS = os.getenv("SHIP_PRODUCTS")
INPUTS = os.getenv("SHIP_INPUTS")
MAX_RETRIES = 100
check = True
final = True

In [ ]:
# ---------- 預先準備 ----------
service = Service("chromedriver.exe")
options = Options()
driver = webdriver.Chrome(service=service, options=options)

# 等待指定時間
print(f"⏳ 等待到 {TARGET_TIME} 開始...")
target_dt = datetime.fromisoformat(TARGET_TIME)
while datetime.now() < target_dt:
    time.sleep(0.5)

# 檢查賣場狀態
driver.get(URL)
WebDriverWait(driver, 3).until(
    lambda d: d.execute_script("return document.readyState") == "complete"
)
notified = False

while "此賣場已由賣家自行下架" in driver.page_source:
    if not notified:
        print(f"⚠️ 賣場尚未上架...")
        notified = True
    time.sleep(0.3)
    driver.get(URL)
    WebDriverWait(driver, 3).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )

print(f"✅ 開始購物流程！")

In [ ]:
# ---------- 登入流程 ----------
alertify_elements = driver.find_elements(By.XPATH, '//*[@id="alertify-ok"]')
if alertify_elements:
    print("🔐 必須是會員才可購買...")
    wait_and_click('//*[@id="alertify-ok"]')
else:
    print("🔐 等待登入按鈕出現並點擊...")
    driver.execute_script("window.scrollBy(0, 500);")  # 滾動讓按鈕顯示
    time.sleep(0.3)
    driver.execute_script("window.scrollBy(0, -500);")
    wait_and_click('//*[@data-target="#loginModal"]')
print("🔐 點擊登入按鈕")
wait_and_click('//*[@title="使用 LINE 帳戶登入"]')
print("🔗 點擊 LINE 登入")

# 輸入帳密並登入
print("📝 輸入帳號密碼登入...")
WebDriverWait(driver, 5).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@placeholder="電子郵件帳號"]'))
)
driver.find_element(By.XPATH, '//*[@placeholder="電子郵件帳號"]').send_keys(EMAIL)
driver.find_element(By.XPATH, '//*[@placeholder="密碼"]').send_keys(PASSWORD)
wait_and_click('//*[text()="登入"]')
print("✅ 登入")

# 等頁面載入完成
WebDriverWait(driver, 5).until(
    lambda d: d.execute_script("return document.readyState") == "complete"
)

In [ ]:
# ---------- 嘗試商品可點擊（最多重試 N 次） ----------
for attempt in range(MAX_RETRIES):
    try:
        print(f"🔄 第 {attempt + 1} 次嘗試...")

        WebDriverWait(driver, 3).until(
            lambda d: d.find_elements(By.CSS_SELECTOR, "[data-prod-name]")
        )
        if "spec" in PRODUCTS[0]:
            spec_element = fuzzy_match(
                "data-prod-name",
                PRODUCTS[0]["name"],
                "data-spec-name",
                PRODUCTS[0]["spec"],
                False,
            )
            WebDriverWait(driver, 3).until(EC.element_to_be_clickable(spec_element))
        else:
            add_btn = fuzzy_match(
                "data-prod-name",
                PRODUCTS[0]["name"],
                "onclick",
                "addToCart(this)",
                False,
            )
            WebDriverWait(driver, 3).until(EC.element_to_be_clickable(add_btn))
        print("🎯 商品按鈕可點擊，準備開始選購")
        break
    except (TimeoutException, ElementClickInterceptedException, NoSuchElementException):
        print("⛔ 尚無法點擊，重新整理中...")
        driver.refresh()
        WebDriverWait(driver, 5).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
else:
    print("❌ 超過最大嘗試次數，仍無法點擊商品")

In [ ]:
# ---------- 加入購物車流程 ----------
error_list = []
for product in PRODUCTS:
    try:
        print(f"🛒 開始處理商品：{product['name']}")

        if "spec" in product:
            fuzzy_match(
                "data-prod-name", product["name"], "data-spec-name", product["spec"]
            )
            print(f"✅ 已選擇規格：{product['spec']}")
        fuzzy_match("data-prod-name", product["name"], "onclick", "addToCart(this)")
        print("✅ 已加入購物車")

        xpath_con = '//*[contains(text(),"繼續選購")]'
        wait_and_click(xpath_con, use_js=True, use_click=False)
        WebDriverWait(driver, 3).until_not(
            EC.visibility_of_element_located((By.XPATH, xpath_con))
        )
        print("➡️ 點擊『繼續選購』\n")
    except Exception as e:
        error_list.append(
            f'{product["name"]}({product["spec"]})'
            if "spec" in product
            else product["name"]
        )
        print(f"❌ 錯誤：處理商品「{product['name']}」時出現問題：{e}\n")

In [ ]:
# ---------- 確認商品 ----------
print(f"✅ 已加入 {len(PRODUCTS)-len(error_list)} / {len(PRODUCTS)} 項商品")
if error_list:
    print("❌ 待處理商品：")
    for product in error_list:
        print(product)

wait_and_click('//*[contains(@class,"cart_trigger")]')
print("\n🛒 點擊購物車")

if check:
    input("確認商品...")

wait_and_click('//*[@onclick="createCart()"]', use_js=True)
print("✅ 點擊『我要結帳』")

In [ ]:
# ---------- 結帳流程 ----------
print("💳 開始結帳流程")
wait_and_click('//label[@for="Agree"]', use_js=True, use_click=False)
print("✅ 勾選購物須知")
wait_and_click('//*[@id="btnNext"]')
print("➡️ 點擊下一步")

if INPUTS:
    # 輸入自訂欄位
    for i, x in enumerate(INPUTS):
        input_field = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located(
                (By.XPATH, f'//input[@name="txtAnswer_{i}"]')
            )
        )
        input_field.clear()
        input_field.send_keys(x)
        print(f"📝 自訂欄位 {i + 1}: {x}")
else:
    input("輸入自訂欄位...")

# 立即結帳
wait_and_click('//*[@id="btn_book"]')
print("✅ 點擊立即結帳")

# 選擇門市
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, "myModal")))
print("📍 門市選擇視窗已出現")
store_radio = '//span[normalize-space()="' + SHOP + '門市"]/ancestor::label'
wait_and_click(store_radio, use_js=True, use_click=False)
print("🏪 選擇『" + SHOP + "門市』")
wait_and_click('//*[@onclick="jsAddress(1)"]', use_js=True)
print("✅ 確認取貨門市")

# 最後送出
if final:
    final_next = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "btnNext"))
    )
    final_next.click()
    print("🚀 提交訂單完成")